In [116]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
import pandas as pd 
import numpy as np
initial_data = pd.read_csv("/content/drive/My Drive/CA682 - Data Management and Visualisation - Suzanne Little/Assignemt 01 | Dec 12/Dataset/2003_to_2018-Initial-1080476.csv")

In [118]:
final_data = pd.read_csv("/content/drive/My Drive/CA682 - Data Management and Visualisation - Suzanne Little/Assignemt 01 | Dec 12/Dataset/2003_to_2018_Final.csv")

In [119]:
latest_data = pd.read_csv("/content/drive/My Drive/CA682 - Data Management and Visualisation - Suzanne Little/Assignemt 01 | Dec 12/Dataset/2018_to_Present.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.



# Data cleaning

In [120]:
latest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260045 entries, 0 to 260044
Data columns (total 36 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   Incident Datetime                                     260045 non-null  object 
 1   Incident Date                                         260045 non-null  object 
 2   Incident Time                                         260045 non-null  object 
 3   Incident Year                                         260045 non-null  int64  
 4   Incident Day of Week                                  260045 non-null  object 
 5   Report Datetime                                       260045 non-null  object 
 6   Row ID                                                260045 non-null  int64  
 7   Incident ID                                           260045 non-null  int64  
 8   Incident Number                             

In [121]:
len(pd.unique(latest_data['Row ID']))

260045

In [122]:
#Concatinating the two dataframes
frame = [initial_data, final_data]
till_2018 = pd.concat(frame)
till_2018.head()

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
0,11087941463010,110879414,63010,WARRANTS,WARRANT ARREST,Friday,07/01/2011,08:00,NORTHERN,"ARREST, BOOKED",3400 Block of PIERCE ST,-122.439758,37.802151,POINT (-122.43975785707501 37.8021507619169),17.0,4.0,6.0,13.0,13.0,9.0,1.0,57.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,17.0
1,5117770316010,51177703,16010,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Tuesday,10/18/2005,14:30,TENDERLOIN,"ARREST, BOOKED",0 Block of UNITEDNATIONS PZ,-122.414318,37.779944,POINT (-122.414317857881 37.7799444052046),21.0,5.0,10.0,36.0,7.0,10.0,9.0,28852.0,36.0,3.0,1.0,3.0,1.0,3.0,6.0,2.0,1.0,1.0,1.0,39.0,21.0
2,5011209707055,50112097,7055,VEHICLE THEFT,RECOVERED VEHICLE - STOLEN OUTSIDE SF,Saturday,01/29/2005,13:45,BAYVIEW,NONE,1500 Block of KIRKWOOD AV,-122.388799,37.737576,POINT (-122.38879889515101 37.7375755833256),86.0,2.0,9.0,1.0,10.0,3.0,8.0,58.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,86.0
3,11044468164020,110444681,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,06/02/2011,02:52,CENTRAL,PSYCHOPATHIC CASE,900 Block of COLUMBUS AV,-122.414354,37.803109,POINT (-122.41435430115101 37.8031089840376),107.0,6.0,3.0,32.0,3.0,1.0,10.0,308.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,107.0
4,3038311107024,30383111,7024,VEHICLE THEFT,STOLEN TRAILER,Saturday,02/01/2003,08:00,BAYVIEW,NONE,1500 Block of BAY SHORE BL,-122.401097,37.724556,POINT (-122.401096851568 37.7245556697717),86.0,2.0,9.0,1.0,10.0,3.0,8.0,58.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,86.0


In [123]:
len(pd.unique(till_2018['IncidntNum']))

1717518

Using only the first occurence of the Incident numbers for Task 1 as it repesent the first reporting of the incidennt and also the duplicates which are present in the table are used to update the incidents which were previously reported.

Keeping only the two columns "IncidntNum" and "DayOfWeek" for task 1 and dropping of the rest.

In [124]:
task1 = till_2018.drop_duplicates(subset = ["IncidntNum"])
task1 = task1[['IncidntNum', 'DayOfWeek']].copy()
task1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1717518 entries, 0 to 1080476
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   IncidntNum  int64 
 1   DayOfWeek   object
dtypes: int64(1), object(1)
memory usage: 39.3+ MB


Doing the same with the "latest_data" dataframe. We will be filtering the uniquie incidents using the ".contains" function. In this dataset "Report Type Description" specifies if the row is an update for an already existing incident or a unique incident.

This is done by appending "Supplement" at the end of the Report Type. Ex. "Coplogic Initial" is a first reported incident and "Coplogic Supplement" is an update related to an already existing incident.

After filtering the unique incidents, only two columns will be kept for task 1 i.e, "Incident Number" and "Incident Day of Week"

In [125]:
dummy = latest_data[latest_data['Report Type Description'].str.contains("Initial")]
dummy.info()
len(pd.unique(dummy['Row ID']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244679 entries, 1 to 260044
Data columns (total 36 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   Incident Datetime                                     244679 non-null  object 
 1   Incident Date                                         244679 non-null  object 
 2   Incident Time                                         244679 non-null  object 
 3   Incident Year                                         244679 non-null  int64  
 4   Incident Day of Week                                  244679 non-null  object 
 5   Report Datetime                                       244679 non-null  object 
 6   Row ID                                                244679 non-null  int64  
 7   Incident ID                                           244679 non-null  int64  
 8   Incident Number                             

244679

Considering Row ID instead of Incident ID was a concious decision made after reading the data carefully. According to the description of the data, two of more incidents can have same "Incident ID", "Incident Number", and "CAD Numbers" and different "Incident Codes".

This occurs when an officer may have a warrant for an arrest and while making the arrest, discovers narcotics in the individual’s possession. The officer would record two Incident Codes: (1) for the warrant and (2) for the discovery of narcotics. And therefore these will still be considered unique incidents.

In [126]:
dummy = dummy[['Row ID', 'Incident Day of Week']].copy()

Renaming the dataframes and merging them for final dataframe which will be used for task1

In [127]:
dummy.rename(columns = {'Row ID':'IncidntNum', 'Incident Day of Week':'DayOfWeek'}, inplace = True)

In [128]:
frame = [task1, dummy]
task1 = pd.concat(frame)
task1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1962197 entries, 0 to 260044
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   IncidntNum  int64 
 1   DayOfWeek   object
dtypes: int64(1), object(1)
memory usage: 44.9+ MB


# What day of week are the officers the busiest?


In [129]:
import plotly
import plotly.express as px

In [130]:
#Getting Frequencies for each day of the week
test1234 = task1.groupby(['DayOfWeek']).agg(['count'])
test1234 = test1234.reset_index()
#Renaming the Columns 
test1234.columns = ["DayOfWeek", "Count"]
test1234
task1 = test1234

In [149]:
# Manually assigning colors to the days.
colors = {"Sunday": '#ffb55f','Monday': '#787878','Tuesday':'#787878', 
          'Wednesday': '#787878', 'Thursday': '#787878', 'Friday': '#d0d2d3', 'Saturday':'#787878'}
fig1 = px.bar(task1, x='DayOfWeek', y='Count', color='DayOfWeek',
              labels={
                     "DayOfWeek": "Days Of Week",
                      "Count" : "Number of Incidents Reported",
                          },
              title={
                     'text': '<b>Number of incidents reported every day of the week since 2003</b>'
                          },
              color_discrete_map={'Sunday': 'rgba(66, 114, 146, 1)',
                                       'Monday': 'rgba(208, 210, 211, 1)',
                                       'Tuesday': 'rgba(208, 210, 211, 1)',
                                       'Wednesday':'rgba(208, 210, 211, 1)',
                                       'Thursday':'rgba(208, 210, 211, 1)',
                                       'Friday':'rgba(0, 76, 109, 1)',
                                       'Saturday':'rgba(208, 210, 211, 1)'},
                   category_orders={ # replaces default order by column name
                        "DayOfWeek": ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
                        }                  
                 )
fig1.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
},
title = dict(
    font = dict(
        size=25
    )
))
fig1.show()


# What type of crimes are being reported across the city.

In [132]:
task2_latest = latest_data
task2_latest.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Y,X,point,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,HSOC Zones as of 2018-06-05,OWED Public Spaces,Central Market/Tenderloin Boundary Polygon - Updated,Parks Alliance CPSI (27+TL sites),ESNCAG - Boundary File,"Areas of Vulnerability, 2016"
0,2020/08/15 08:56:00 AM,2020/08/15,08:56,2020,Saturday,2020/08/15 08:56:00 AM,95300907041,953009,200474239,NaN,VS,Vehicle Supplement,NaN,7041,RECOVERED VEHICLE,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020/08/15 09:40:00 AM,2020/08/15,09:40,2020,Saturday,2020/08/15 06:21:00 PM,95322706244,953227,206121692,NaN,II,Coplogic Initial,True,6244,LARCENY/THEFT,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,NaN,NaN,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018/02/24 10:00:00 PM,2018/02/24,22:00,2018,Saturday,2018/03/02 10:13:00 AM,64174871000,641748,186051531,NaN,II,Coplogic Initial,True,71000,STOLEN PROPERTY,Lost Property,Lost Property,Open or Active,NaN,NaN,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020/08/16 03:13:00 AM,2020/08/16,03:13,2020,Sunday,2020/08/16 03:14:00 AM,95319604083,953196,200491669,202290313.0,II,Initial,NaN,4083,ASSAULT,Aggravated Assault,"Firearm, Discharging in Grossly Negligent Manner",Open or Active,23RD ST \ ARKANSAS ST,23642000.0,Bayview,Potrero Hill,10.0,37.754827,-122.397729,"(37.75482657770952, -122.39772873392515)",54.0,2,9.0,26.0,NaN,NaN,NaN,NaN,NaN,2.0
4,2020/08/16 03:38:00 AM,2020/08/16,03:38,2020,Sunday,2020/08/16 04:56:00 AM,95326228100,953262,200491738,202290404.0,II,Initial,NaN,28100,VANDALISM,Vandalism,"Malicious Mischief, Breaking Windows",Open or Active,VALENCIA ST \ 15TH ST,24377000.0,Mission,Mission,9.0,37.766540,-122.422044,"(37.76653957529556, -122.42204381448558)",53.0,3,2.0,20.0,3.0,NaN,NaN,NaN,NaN,2.0


In [133]:
#Selecting only the incidents reported after 2019
task2_latest = task2_latest.loc[(task2_latest['Incident Datetime'].str[:4]).astype(int) > 2019] 
task2_latest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36590 entries, 0 to 38292
Data columns (total 36 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Incident Datetime                                     36590 non-null  object 
 1   Incident Date                                         36590 non-null  object 
 2   Incident Time                                         36590 non-null  object 
 3   Incident Year                                         36590 non-null  int64  
 4   Incident Day of Week                                  36590 non-null  object 
 5   Report Datetime                                       36590 non-null  object 
 6   Row ID                                                36590 non-null  int64  
 7   Incident ID                                           36590 non-null  int64  
 8   Incident Number                                       36

In [134]:
task2_latest.rename(columns = {'Incident Category':'Category', 'X':'lon', 'Y':'lat'}, inplace = True)
#Selecting only the required columns.
task2_latest = task2_latest[['Category', 'lon', 'lat', 'Incident Description']].copy()

task2 = task2_latest
task2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36590 entries, 0 to 38292
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Category              36590 non-null  object 
 1   lon                   34638 non-null  float64
 2   lat                   34638 non-null  float64
 3   Incident Description  36590 non-null  object 
dtypes: float64(2), object(2)
memory usage: 1.4+ MB


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
#Getting top 5 incident category according to numbers
dummy12 = task2.groupby(['Category']).agg(['count'])
dummy12 = dummy12.reset_index()
dummy12.columns = ["Category", "lon_count", "lat_count", "Incident Description"]
dummy12 = dummy12.sort_values('lon_count')
dummy12.tail(5)

,Category,lon_count,lat_count,Incident Description
15,Other Miscellaneous,2482,2482,2488
25,VEHICLE THEFT,2490,2490,2535
1,BURGLARY,2883,2883,2915
24,VANDALISM,2901,2901,3000
10,LARCENY/THEFT,7969,7969,8723


In [136]:
#task2_latest["Incident Category"].unique() 
task2 = task2.loc[task2['Category'].isin(['BURGLARY', 'VANDALISM', 'LARCENY/THEFT',
                                          'VEHICLE THEFT', 'Other Miscellaneous'])]

In [137]:
import plotly.graph_objs as go
import plotly

task2["Category"].unique()

task2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19661 entries, 1 to 38287
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Category              19661 non-null  object 
 1   lon                   18725 non-null  float64
 2   lat                   18725 non-null  float64
 3   Incident Description  19661 non-null  object 
dtypes: float64(2), object(2)
memory usage: 768.0+ KB


In [152]:
fig2 = px.scatter_mapbox(task2, lat="lat", lon="lon", hover_name="Category",
                         hover_data=["Incident Description"],
                        color='Category', zoom=10.45, height=350)
fig2.update_layout(mapbox_style="open-street-map")
fig2.update_layout(margin={"r":0,"t":55,"l":0,"b":0})
fig2.update_layout(title='<b>Top 5 reported crimes accross the city after 2019</b>')
fig2.update_layout(
title = dict(
    font = dict(
        size=25
    )
))
fig2.show()

# Busiest police district over the years







3 columns: one corresponding to the items you are ranking (Item in our case), one corresponding to the value by which you are ranking (Value in our case), and another corresponding to the year or date of each corresponding item-value pair (Year in our case)

In [139]:
c1_task3 = till_2018
c2_task3 = latest_data

c1_task3 = till_2018.drop_duplicates(subset = ["IncidntNum"])
#Selecting only the year from 'Date' column and storing it in new column 'Year'
c1_task3['Year'] = c1_task3['Date'].str[-4:]
c1_task3 = c1_task3[['IncidntNum', 'PdDistrict', 'Year']].copy()
u_c1_task3 = c1_task3.groupby(['Year','PdDistrict']).agg(['count'])
u_c1_task3 = u_c1_task3.reset_index()
u_c1_task3.columns = ["Year", "Item", "Value"]
#u_c1_task3.head()

c2_task3 = till_2018.drop_duplicates(subset = ["IncidntNum"])
#Selecting only the year from 'Date' column and storing it in new column 'Year'
c2_task3['Year'] = latest_data['Incident Datetime'].str[:4]
c2_task3 = c2_task3[['IncidntNum', 'PdDistrict', 'Year']].copy()
u_c2_task3 = c2_task3.groupby(['Year','PdDistrict']).agg(['count'])
u_c2_task3 = u_c2_task3.reset_index()
u_c2_task3.columns = ["Year", "Item", "Value"]
u_c2_task3.head()

frame = [u_c1_task3, u_c2_task3]
data = pd.concat(frame)
data['Year'] = data['Year'].astype(int)
data.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    190 non-null    int64 
 1   Item    190 non-null    object
 2   Value   190 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.9+ KB


In [140]:
import plotly.graph_objects as go
fig = {
    "data": [],
    "layout": {},
    "frames": []
}

In [141]:
#assigning colors to the categories.
colors = {'CENTRAL': 'rgb(208, 210, 211)',
          'BAYVIEW': 'rgb(208, 210, 211)',
          'NORTHERN': 'rgba(66, 114, 146, 1)',
          'MISSION':'rgba(40, 95, 127, 1)',
          'SOUTHERN':'rgba(0, 76, 109, 1)',
          'INGLESIDE':'rgb(208, 210, 211)',
          'TENDERLOIN':'rgb(208, 210, 211)',
          'TARAVAL':'rgb(208, 210, 211)',
          'RICHMOND':'rgb(208, 210, 211)',
          'PARK':'rgb(208, 210, 211)'}
data['color'] = data['Item'].map(colors)

data.head()

,Year,Item,Value,color
0,2003,BAYVIEW,11918,"rgb(208, 210, 211)"
1,2003,CENTRAL,11237,"rgb(208, 210, 211)"
2,2003,INGLESIDE,10446,"rgb(208, 210, 211)"
3,2003,MISSION,16271,"rgba(40, 95, 127, 1)"
4,2003,NORTHERN,15136,"rgba(66, 114, 146, 1)"


In [142]:
#2 Slice data, select earliest date available

##2.1 Select earliest year available 
frame1 = data[data['Year'] == data['Year'].min()]

##2.2 Sort the data by value and get the top 10 entries
top_entries = 10 # modify to get top 5, top 20 or any other
frame1 = frame1.sort_values('Value', ascending = False).iloc[:top_entries, :]

##2.3 Sort values in ascending order so that top bar corresponds to greatest value
frame1 = frame1.sort_values('Value', ascending = True)

#3 Populate data entry in plotly figure with frame1
fig['data'] = go.Bar(
  x = frame1['Value'],
  y = frame1['Item'],
  marker_color = frame1['color'],
  hoverinfo = 'all', # display Item name and Value when hovering over bar
  textposition = 'outside', # position text outside bar
  texttemplate = '%{y}<br>%{x:.4s}', # display y field (Item name), line break and then the value with up to 4 digits
  orientation = 'h' # to have bar growing rightwards
)

In [143]:
fig['layout'] = go.Layout(
  font = {'size' : 14},
  plot_bgcolor = 'white',
  xaxis = {'showline': True,
           'visible': True,
           'range': (0, data['Value'].max()),
           'title_text': 'Number of reported Crimes' # title of the x axis
          },
  yaxis = {'showline': False, 
          'visible': True, # to show the title
           'showticklabels': False, # to avoid displaying the names
          'title_text': 'Top ' + str(top_entries) + ' Police Districts'}, # title of the y axis
  bargap = 0.15, # number btwn 0 and 1, sets gap between adjacent bars (as plot fraction)
  title = 'Number of cases reported to Top 10 Police Districts over time' # sets title for default plot
)

In [144]:
fig["layout"]["updatemenus"] = [
    {
        "buttons": [ # define our buttons: Play and Pause
            {
                "args": [None, {"frame": {"duration": 500 # how long the frame is still (in milliseconds)
                                          , "redraw": False}, # for optimised rendering
                                "fromcurrent": True, 
                                "transition": {"duration": 300, # duration of the transition btwn frames
                                               "easing": "quadratic-in-out"}}],
                "label": "Play", # label to be displayed
                "method": "animate" # other options are relayout, restyle and update.
            },
            {
                "args": [[None], {"frame": {"duration": 0,
                                            "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
      # for left-to-right appearance, 
      # change to 'up' for top-to-bottom appearance
        "direction": "left", 
      # right and top padding 
        "pad": {"r": 10, "t": 87},
      # whether to display button in shaded color when pressed
        "showactive": True, 
        "type": "buttons", # specify we are using buttons and not dropdown
      # position in figure (relative)
        "x": 0.1,
        "y": 0
    }
]

In [145]:
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

In [146]:
for year in range(data['Year'].min(), data['Year'].max()+1):

    #1 Same steps than what what we did with frame1
    ##.1 filter out by year
    snap_data = data[data['Year'] == year]

    ##.2 get_top 10
    snap_data = snap_data.sort_values('Value', ascending=False).iloc[:top_entries,:]

    ##.3 get top enttry at top of chart
    snap_data = snap_data.sort_values('Value', ascending=True)

    #2 make frame
    ##.1
    frame = go.Frame(
        data=[
            go.Bar(
                x=snap_data['Value'],
                y=snap_data['Item'],
                marker_color=snap_data['color'],
                hoverinfo='all',
                textposition='outside',
                texttemplate= '%{y}<br>%{x:.4s}',
                orientation = 'h'
            )
        ],
        layout=go.Layout(
            font={'size': 14},
            plot_bgcolor = 'white',
            xaxis={'showline': True,
                   'visible': True,
                   'range': (0, data['Value'].max())
                  },
            yaxis={'showline': False,
                   'visible': False
                  },
            bargap= 0.15,
            title= 'Top 10 Crops across time'
        ),
        # very important to set name of the frame 
        # so that it matches with the slider_step next
        name = year
    )


    ##.2 Append frame to list of frames
    fig['frames'].append(frame)

    # 3 Define slider step so that is linked to the latest frame

    ##.1 define slider step
    slider_step = {"args": [
        [year], # this one needs to match the frame name
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
                   "label": year, # This one defines the sliders tick names
                   "method": "animate"} # same as with buttons
    ##.2 Append to sliders_dict
    sliders_dict["steps"].append(slider_step)

    ### Note for the keen reader ### 
    # frame duration and transition duration do not affect the play and pause buttons,
    # they affect the times when you move the slider manually and then press play again

    # 4 Modify layout sliders of our figure 
    fig['layout']['sliders'] = [sliders_dict]

In [150]:
go.Figure(fig)

# References

1. As I was new to Plotly I made use of their documentation to figure out it how it worked. (https://plotly.com/python/)
2. To understand the concept of colors to be used for visualisation I made use of these two articles: 
- https://www.dataquest.io/blog/what-to-consider-when-choosing-colors-for-data-visualization/ 
- https://chartio.com/learn/charts/how-to-choose-colors-data-visualization/
3. For my last visualization I referred to this article and made appropriate changes according to my requirements to get the desired output. https://towardsdatascience.com/making-a-bar-chart-race-plot-using-plotly-made-easy-8dad3b1da955